# TV Script Generation
In this project, you'll generate your own [Simpsons](https://en.wikipedia.org/wiki/The_Simpsons) TV scripts using RNNs.  You'll be using part of the [Simpsons dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data) of scripts from 27 seasons.  The Neural Network you'll build will generate a new TV script for a scene at [Moe's Tavern](https://simpsonswiki.com/wiki/Moe's_Tavern).
## Get the Data
The data is already provided for you.  You'll be using a subset of the original dataset.  It consists of only the scenes in Moe's Tavern.  This doesn't include other versions of the tavern, like "Moe's Cavern", "Flaming Moe's", "Uncle Moe's Family Feed-Bag", etc..

In [24]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper

data_dir = './data/simpsons/moes_tavern_lines.txt'
text = helper.load_data(data_dir)
# Ignore notice, since we don't use it for analysing the data
text = text[81:]

## Explore the Data
Play around with `view_sentence_range` to view different parts of the data.

In [25]:
view_sentence_range = (0, 10)
print("overall length",len(text))

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))
scenes = text.split('\n\n')
print('Number of scenes: {}'.format(len(scenes)))
sentence_count_scene = [scene.count('\n') for scene in scenes]
print('Average number of sentences in each scene: {}'.format(np.average(sentence_count_scene)))

sentences = [sentence for scene in scenes for sentence in scene.split('\n')]
print('Number of lines: {}'.format(len(sentences)))
word_count_sentence = [len(sentence.split()) for sentence in sentences]
print('Average number of words in each line: {}'.format(np.average(word_count_sentence)))

print()
print('The sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

overall length 305189
Dataset Stats
Roughly the number of unique words: 11492
Number of scenes: 262
Average number of sentences in each scene: 15.248091603053435
Number of lines: 4257
Average number of words in each line: 11.50434578341555

The sentences 0 to 10:
Moe_Szyslak: (INTO PHONE) Moe's Tavern. Where the elite meet to drink.
Bart_Simpson: Eh, yeah, hello, is Mike there? Last name, Rotch.
Moe_Szyslak: (INTO PHONE) Hold on, I'll check. (TO BARFLIES) Mike Rotch. Mike Rotch. Hey, has anybody seen Mike Rotch, lately?
Moe_Szyslak: (INTO PHONE) Listen you little puke. One of these days I'm gonna catch you, and I'm gonna carve my name on your back with an ice pick.
Moe_Szyslak: What's the matter Homer? You're not your normal effervescent self.
Homer_Simpson: I got my problems, Moe. Give me another one.
Moe_Szyslak: Homer, hey, you should not drink to forget your problems.
Barney_Gumble: Yeah, you should only drink to enhance your social skills.




In [26]:
%load_ext autoreload
%autoreload 2
# reoad modules before executing user code.Useful to play around with the tests

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Implement Preprocessing Functions
The first thing to do to any dataset is preprocessing.  Implement the following preprocessing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, you first need to transform the words to ids.  In this function, create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Return these dictionaries in the following tuple `(vocab_to_int, int_to_vocab)`

In [27]:
import numpy as np
import problem_unittests as tests

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    vocab=list(set(text))
    index=range(len(vocab))
    vocab_to_int=dict(zip(vocab,index))
    int_to_vocab=dict(zip(index,vocab))
    return vocab_to_int,int_to_vocab

sample=text[:100].split(" ")
print("Sample to encode:\n", sample)
print("Vocab_to_int:%s\nInt_to_vocab:%s" % create_lookup_tables(sample))
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_create_lookup_tables(create_lookup_tables)

Sample to encode:
 ['Moe_Szyslak:', '(INTO', 'PHONE)', "Moe's", 'Tavern.', 'Where', 'the', 'elite', 'meet', 'to', 'drink.\nBart_Simpson:', 'Eh,', 'yeah,', 'hello']
Vocab_to_int:{'Tavern.': 0, "Moe's": 1, 'to': 2, 'Where': 3, 'elite': 4, 'drink.\nBart_Simpson:': 5, 'Eh,': 6, 'PHONE)': 7, 'hello': 8, 'Moe_Szyslak:': 9, 'yeah,': 10, '(INTO': 11, 'the': 12, 'meet': 13}
Int_to_vocab:{0: 'Tavern.', 1: "Moe's", 2: 'to', 3: 'Where', 4: 'elite', 5: 'drink.\nBart_Simpson:', 6: 'Eh,', 7: 'PHONE)', 8: 'hello', 9: 'Moe_Szyslak:', 10: 'yeah,', 11: '(INTO', 12: 'the', 13: 'meet'}
Tests Passed


### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks make it hard for the neural network to distinguish between the word "bye" and "bye!".

Implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Create a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( . )
- Comma ( , )
- Quotation Mark ( " )
- Semicolon ( ; )
- Exclamation mark ( ! )
- Question mark ( ? )
- Left Parentheses ( ( )
- Right Parentheses ( ) )
- Dash ( -- )
- Return ( \n )

This dictionary will be used to token the symbols and add the delimiter (space) around it.  This separates the symbols as it's own word, making it easier for the neural network to predict on the next word. Make sure you don't use a token that could be confused as a word. Instead of using the token "dash", try using something like "||dash||".

In [28]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenize dictionary where the key is the punctuation and the value is the token
    """
    to_replace={
       '.':'||Period||', 
       ',':'||Comma||', 
       '"':'||Quotation||', 
       ';':'||Semicolon||', 
       '!':'||ExclamationMark||', 
       '(':'||LParentheses||', 
       ')':'||RParentheses||', 
       '--':'||Dash||', 
       '\n':'||Return||'  ,
        '?':'||QuestionMark||'       

    }
    return to_replace

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_tokenize(token_lookup)

Tests Passed


## Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [29]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Preprocess Training, Validation, and Testing Data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [30]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import numpy as np
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

## Build the Neural Network
You'll build the components necessary to build a RNN by implementing the following functions below:
- get_inputs
- get_init_cell
- get_embed
- build_rnn
- build_nn
- get_batches

### Check the Version of TensorFlow and Access to GPU

In [31]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.0.0


/home/lina/bin/anaconda3/envs/udacity-tv-script/lib/python3.6/site-packages/ipykernel/__main__.py:14: UserWarning: No GPU found. Please use a GPU to train your neural network.


### Input
Implement the `get_inputs()` function to create TF Placeholders for the Neural Network.  It should create the following placeholders:
- Input text placeholder named "input" using the [TF Placeholder](https://www.tensorflow.org/api_docs/python/tf/placeholder) `name` parameter.
- Targets placeholder
- Learning Rate placeholder

Return the placeholders in the following tuple `(Input, Targets, LearningRate)`

In [32]:
def get_inputs():
    """
    Create TF Placeholders for input, targets, and learning rate.
    :return: Tuple (input, targets, learning rate)
    """
    inputs=tf.placeholder(dtype=tf.int32,name="input", shape=(None,None)) # Batchsize x Sequence Length
    targets=tf.placeholder(dtype=tf.int32,name="target",shape=(None,None)) # Batchsize x Sequence Length
    learning_rate=tf.placeholder(dtype=tf.float32,name="learning_rate")
    return inputs,targets,learning_rate

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_inputs(get_inputs)

Tests Passed


### Build RNN Cell and Initialize
Stack one or more [`BasicLSTMCells`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/BasicLSTMCell) in a [`MultiRNNCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell).
- The Rnn size should be set using `rnn_size`
- Initalize Cell State using the MultiRNNCell's [`zero_state()`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell#zero_state) function
    - Apply the name "initial_state" to the initial state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the cell and initial state in the following tuple `(Cell, InitialState)`

In [33]:
def get_init_cell(batch_size, rnn_size):
    """
    Create an RNN Cell and initialize it.
    :param batch_size: Size of batches
    :param rnn_size: Size of RNNs
    :return: Tuple (cell, initialize state)
    """
    num_layers=1
    cell = tf.contrib.rnn.BasicLSTMCell(rnn_size)
    stacked_lstm = tf.contrib.rnn.MultiRNNCell([cell] * num_layers)
    initial_state = stacked_lstm.zero_state(batch_size, tf.float32)
    initial_state=tf.identity(initial_state, name="initial_state")
    
    return stacked_lstm, initial_state
 
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_init_cell(get_init_cell)

Tests Passed


### Word Embedding
Apply embedding to `input_data` using TensorFlow.  Return the embedded sequence.

In [34]:
def get_embed(input_data, vocab_size, embed_dim):
    """
    Create embedding for <input_data>.
    :param input_data: TF placeholder for text input.
    :param vocab_size: Number of words in vocabulary.
    :param embed_dim: Number of embedding dimensions
    :return: Embedded input.
    """
    embedding = tf.get_variable("embedding",shape=[vocab_size, embed_dim])
    inputs = tf.nn.embedding_lookup(embedding, input_data)    
    return inputs
 
# add additional test     
def test_input_out_of_vocab_size(get_embed):
    with tf.Graph().as_default() as g:
        test_input_data = tf.placeholder(tf.int32, [2,3]) # numsequences x num characters
        test_vocab_size = 27
        test_embed_dim = 256
        embed = get_embed(test_input_data, test_vocab_size, test_embed_dim)
        sess = tf.InteractiveSession(graph=g)
        sess.run(tf.global_variables_initializer())
        feed = {test_input_data: [[1,2,3],[1,2,26]]} # all indices within test_vocab_size
        sess.run(embed, feed)
        try:
            feed = {test_input_data: [[1,2,3],[1,2,32]]} # 32 is not within test_vocab_size

            print(sess.run(embed, feed))
        except tf.errors.InvalidArgumentError:
            print("additional test passed")
          
test_input_out_of_vocab_size(get_embed)
    
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_embed(get_embed)

additional test passed
Tests Passed


### Build RNN
You created a RNN Cell in the `get_init_cell()` function.  Time to use the cell to create a RNN.
- Build the RNN using the [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)
 - Apply the name "final_state" to the final state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the outputs and final_state state in the following tuple `(Outputs, FinalState)` 

In [12]:
def build_rnn(cell, inputs):
    """
    Create a RNN using a RNN Cell
    :param cell: RNN Cell
    :param inputs: Input text data
    :return: Tuple (Outputs, Final State)
    """
    # 'outputs' is a tensor of shape [batch_size, max_time, rnn_size]
    # 'state' is a N-tuple where N is the number of LSTMCells containing a
    # tf.contrib.rnn.LSTMStateTuple for each cell
    outputs, state = tf.nn.dynamic_rnn(cell, inputs,dtype=tf.float32)
    state=tf.identity(state, name="final_state")
    return  outputs, state

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_rnn(build_rnn)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:
- Apply embedding to `input_data` using your `get_embed(input_data, vocab_size, embed_dim)` function.
- Build RNN using `cell` and your `build_rnn(cell, inputs)` function.
- Apply a fully connected layer with a linear activation and `vocab_size` as the number of outputs.

Return the logits and final state in the following tuple (Logits, FinalState) 

In [35]:
def build_nn(cell, rnn_size, input_data, vocab_size, embed_dim):
    """
    Build part of the neural network
    :param cell: RNN cell
    :param rnn_size: Size of rnns
    :param input_data: Input data
    :param vocab_size: Vocabulary size
    :param embed_dim: Number of embedding dimensions
    :return: Tuple (Logits, FinalState)
    """
    inputs=get_embed(input_data, vocab_size, embed_dim)
    outputs, final_state=build_rnn(cell, inputs)
    logits= tf.contrib.layers.fully_connected(outputs,num_outputs=vocab_size,activation_fn=None)
        
    return logits, final_state


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_nn(build_nn)

Tests Passed


### Batches
Implement `get_batches` to create batches of input and targets using `int_text`.  The batches should be a Numpy array with the shape `(number of batches, 2, batch size, sequence length)`. Each batch contains two elements:
- The first element is a single batch of **input** with the shape `[batch size, sequence length]`
- The second element is a single batch of **targets** with the shape `[batch size, sequence length]`

If you can't fill the last batch with enough data, drop the last batch.

For example, `get_batches([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 3, 2)` would return a Numpy array of the following:
```
[
  # First Batch
  [
    # Batch of Input
    [[ 1  2], [ 7  8], [13 14]]
    # Batch of targets
    [[ 2  3], [ 8  9], [14 15]]
  ]

  # Second Batch
  [
    # Batch of Input
    [[ 3  4], [ 9 10], [15 16]]
    # Batch of targets
    [[ 4  5], [10 11], [16 17]]
  ]

  # Third Batch
  [
    # Batch of Input
    [[ 5  6], [11 12], [17 18]]
    # Batch of targets
    [[ 6  7], [12 13], [18  1]]
  ]
]
```

Notice that the last target value in the last batch is the first input value of the first batch. In this case, `1`. This is a common technique used when creating sequence batches, although it is rather unintuitive.

In [36]:
def get_batches(int_text, batch_size, seq_length):
    """
    Return batches of input and target
    :param int_text: Text with the words replaced by their ids
    :param batch_size: The size of batch
    :param seq_length: The length of sequence
    :return: Batches as a Numpy array
    """
    num_char_in_batch=batch_size*seq_length # C
    full_batches=len(int_text)//num_char_in_batch 
    remainder=len(int_text) % num_char_in_batch
    cropped=int_text[:-remainder]
       
    # make as many rows as examples in a batch (batch_size)
    cropped=np.reshape(cropped, (batch_size,-1)) 
    batches=[]
   
    shift=1 #shift for the target
    for n in range(0,cropped.shape[1],seq_length):
        # Slide over the rows, creating one sequence for all rows in each step
        feature = cropped[:, n:n+seq_length] # B x S
        target_tmp = cropped[:, n+shift:n+seq_length+shift] 

        # For the very last batch, the targets will be as many characters short at the end of 
        # the sequences as the shift value. That breaks things 
        # since the target are no longer of the same size across batches. 
        # To get around this, I'll make an array of the appropriate size first, of all zeros, then add the targets.
        # For all batches but the last one that will not change anything
        # The last batch however will be padded with zeros 
        target = np.zeros(feature.shape, dtype=feature.dtype)
        target[:,:target_tmp.shape[1]] = target_tmp 
        batches.append((feature,target))
    return np.array(batches)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_batches(get_batches)

Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `num_epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `embed_dim` to the size of the embedding.
- Set `seq_length` to the length of sequence.
- Set `learning_rate` to the learning rate.
- Set `show_every_n_batches` to the number of batches the neural network should print progress.

In [37]:
# Number of Epochs
num_epochs = 100  # 50 is ok to drop under the required threshold
# Batch Size
batch_size = 128
# RNN Size
rnn_size = 256
# Embedding Dimension Size
embed_dim = 256 
# Sequence Length
seq_length = 25
# Learning Rate
learning_rate = 0.01
# Show stats for every n number of batches
show_every_n_batches = 10


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
save_dir = './save'

### Build the Graph
Build the graph using the neural network you implemented.

In [38]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from tensorflow.contrib import seq2seq

train_graph = tf.Graph()
with train_graph.as_default():
    vocab_size = len(int_to_vocab)
    input_text, targets, lr = get_inputs()
    input_data_shape = tf.shape(input_text)
    cell, initial_state = get_init_cell(input_data_shape[0], rnn_size)
    logits, final_state = build_nn(cell, rnn_size, input_text, vocab_size, embed_dim)

    # Probabilities for generating words
    probs = tf.nn.softmax(logits, name='probs')

    # Loss function
    cost = seq2seq.sequence_loss(
        logits,
        targets,
        tf.ones([input_data_shape[0], input_data_shape[1]]))

    # Optimizer
    optimizer = tf.train.AdamOptimizer(lr)

    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
    train_op = optimizer.apply_gradients(capped_gradients)

## Train
Train the neural network on the preprocessed data.  If you have a hard time getting a good loss, check the [forums](https://discussions.udacity.com/) to see if anyone is having the same problem.

In [23]:
"""
MODIFIED VERSION FOR EXPERIMENTING WITH HYPERPARAMS
"""
import time
batches = get_batches(int_text[:-batch_size*seq_length-1], batch_size, seq_length)
val_batches=get_batches(int_text[-batch_size*seq_length-1:],batch_size, seq_length)

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(num_epochs):
        state = sess.run(initial_state, {input_text: batches[0][0]})
        duration=[]
        #np.random.shuffle(batches)
        for batch_i, (x, y) in enumerate(batches):
            start_time=time.time()
            feed = {
                input_text: x,
                targets: y,
                initial_state: state,
                lr: learning_rate}
            train_loss, state, _ = sess.run([cost, final_state, train_op], feed)
            duration.append(time.time()-start_time)  

            # Show every <show_every_n_batches> batches
            if (epoch_i * len(batches) + batch_i) % show_every_n_batches == 0:
                val_loss=[]
                for (x, y) in val_batches:
                    feed = {
                        input_text: x,
                        targets: y
                        }
                    val_loss.append(sess.run([cost], feed))
                print('Epoch {:>3} Batch {:>4}/{} b_duration = {:>.2f}s  train_loss = {:.3f} validation loss = {:.3f}'.format(
                    epoch_i,
                    batch_i,
                    len(batches),
                    np.average(duration),
                    train_loss,
                    np.average(val_loss)))
        print("Total time for epoch {:.<2f} secs ".format(np.sum(duration)))
        # Save Model
        saver = tf.train.Saver()
        saver.save(sess, save_dir+"_"+str(epoch_i))
        print('Model Trained and Saved')

Epoch   0 Batch    0/20 b_duration = 2.86s  train_loss = 8.822 validation loss = 8.807
Epoch   0 Batch   10/20 b_duration = 2.77s  train_loss = 6.899 validation loss = 6.849
Total time for epoch 57.789757 secs 
Model Trained and Saved
Epoch   1 Batch    0/20 b_duration = 3.50s  train_loss = 6.847 validation loss = 6.595
Epoch   1 Batch   10/20 b_duration = 3.55s  train_loss = 6.276 validation loss = 6.366
Total time for epoch 78.822831 secs 
Model Trained and Saved
Epoch   2 Batch    0/20 b_duration = 4.21s  train_loss = 6.361 validation loss = 6.367
Epoch   2 Batch   10/20 b_duration = 5.36s  train_loss = 6.303 validation loss = 6.360
Total time for epoch 102.627934 secs 
Model Trained and Saved
Epoch   3 Batch    0/20 b_duration = 4.07s  train_loss = 6.274 validation loss = 6.364
Epoch   3 Batch   10/20 b_duration = 4.70s  train_loss = 6.241 validation loss = 6.340
Total time for epoch 100.188702 secs 
Model Trained and Saved
Epoch   4 Batch    0/20 b_duration = 5.57s  train_loss = 6

Model Trained and Saved
Epoch  35 Batch    0/20 b_duration = 3.14s  train_loss = 4.378 validation loss = 5.435
Epoch  35 Batch   10/20 b_duration = 3.20s  train_loss = 4.403 validation loss = 5.425
Total time for epoch 66.205586 secs 
Model Trained and Saved
Epoch  36 Batch    0/20 b_duration = 3.14s  train_loss = 4.283 validation loss = 5.422
Epoch  36 Batch   10/20 b_duration = 3.32s  train_loss = 4.330 validation loss = 5.444
Total time for epoch 66.118347 secs 
Model Trained and Saved
Epoch  37 Batch    0/20 b_duration = 3.21s  train_loss = 4.196 validation loss = 5.426
Epoch  37 Batch   10/20 b_duration = 3.32s  train_loss = 4.238 validation loss = 5.481
Total time for epoch 66.399757 secs 
Model Trained and Saved
Epoch  38 Batch    0/20 b_duration = 3.65s  train_loss = 4.101 validation loss = 5.437
Epoch  38 Batch   10/20 b_duration = 3.44s  train_loss = 4.197 validation loss = 5.500
Total time for epoch 69.724475 secs 
Model Trained and Saved
Epoch  39 Batch    0/20 b_duration =

KeyboardInterrupt: 

In [39]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
batches = get_batches(int_text, batch_size, seq_length)

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(num_epochs):
        state = sess.run(initial_state, {input_text: batches[0][0]})

        for batch_i, (x, y) in enumerate(batches):
            feed = {
                input_text: x,
                targets: y,
                initial_state: state,
                lr: learning_rate}
            train_loss, state, _ = sess.run([cost, final_state, train_op], feed)

            # Show every <show_every_n_batches> batches
            if (epoch_i * len(batches) + batch_i) % show_every_n_batches == 0:
                print('Epoch {:>3} Batch {:>4}/{}   train_loss = {:.3f}'.format(
                    epoch_i,
                    batch_i,
                    len(batches),
                    train_loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_dir)
    print('Model Trained and Saved')

Epoch   0 Batch    0/21   train_loss = 8.822
Epoch   0 Batch   10/21   train_loss = 6.803
Epoch   0 Batch   20/21   train_loss = 6.864
Epoch   1 Batch    9/21   train_loss = 6.049
Epoch   1 Batch   19/21   train_loss = 5.878
Epoch   2 Batch    8/21   train_loss = 5.598
Epoch   2 Batch   18/21   train_loss = 5.297
Epoch   3 Batch    7/21   train_loss = 5.048
Epoch   3 Batch   17/21   train_loss = 4.883
Epoch   4 Batch    6/21   train_loss = 4.799
Epoch   4 Batch   16/21   train_loss = 4.675
Epoch   5 Batch    5/21   train_loss = 4.452
Epoch   5 Batch   15/21   train_loss = 4.406
Epoch   6 Batch    4/21   train_loss = 4.436
Epoch   6 Batch   14/21   train_loss = 4.139
Epoch   7 Batch    3/21   train_loss = 4.145
Epoch   7 Batch   13/21   train_loss = 4.094
Epoch   8 Batch    2/21   train_loss = 4.022
Epoch   8 Batch   12/21   train_loss = 3.911
Epoch   9 Batch    1/21   train_loss = 3.817
Epoch   9 Batch   11/21   train_loss = 3.706
Epoch  10 Batch    0/21   train_loss = 3.698
Epoch  10 

Epoch  87 Batch    3/21   train_loss = 0.267
Epoch  87 Batch   13/21   train_loss = 0.259
Epoch  88 Batch    2/21   train_loss = 0.296
Epoch  88 Batch   12/21   train_loss = 0.250
Epoch  89 Batch    1/21   train_loss = 0.261
Epoch  89 Batch   11/21   train_loss = 0.300
Epoch  90 Batch    0/21   train_loss = 0.309
Epoch  90 Batch   10/21   train_loss = 0.278
Epoch  90 Batch   20/21   train_loss = 0.328
Epoch  91 Batch    9/21   train_loss = 0.309
Epoch  91 Batch   19/21   train_loss = 0.312
Epoch  92 Batch    8/21   train_loss = 0.335
Epoch  92 Batch   18/21   train_loss = 0.360
Epoch  93 Batch    7/21   train_loss = 0.351
Epoch  93 Batch   17/21   train_loss = 0.384
Epoch  94 Batch    6/21   train_loss = 0.376
Epoch  94 Batch   16/21   train_loss = 0.370
Epoch  95 Batch    5/21   train_loss = 0.397
Epoch  95 Batch   15/21   train_loss = 0.425
Epoch  96 Batch    4/21   train_loss = 0.425
Epoch  96 Batch   14/21   train_loss = 0.437
Epoch  97 Batch    3/21   train_loss = 0.460
Epoch  97 

## Save Parameters
Save `seq_length` and `save_dir` for generating a new TV script.

In [40]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params((seq_length, save_dir))

# Checkpoint

In [41]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
seq_length, load_dir = helper.load_params()


## Implement Generate Functions
### Get Tensors
Get tensors from `loaded_graph` using the function [`get_tensor_by_name()`](https://www.tensorflow.org/api_docs/python/tf/Graph#get_tensor_by_name).  Get the tensors using the following names:
- "input:0"
- "initial_state:0"
- "final_state:0"
- "probs:0"

Return the tensors in the following tuple `(InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)` 

In [42]:
def get_tensors(loaded_graph):
    """
    Get input, initial state, final state, and probabilities tensor from <loaded_graph>
    :param loaded_graph: TensorFlow graph loaded from file
    :return: Tuple (InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)
    """
    inputs = loaded_graph.get_tensor_by_name("input:0") 
    initial_state = loaded_graph.get_tensor_by_name("initial_state:0")   
    final_state = loaded_graph.get_tensor_by_name("final_state:0")   
    probabilities = loaded_graph.get_tensor_by_name("probs:0")   

    return inputs, initial_state, final_state, probabilities

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_tensors(get_tensors)

Tests Passed


### Choose Word
Implement the `pick_word()` function to select the next word using `probabilities`.

In [44]:
def pick_word(probabilities, int_to_vocab):
    """
    Pick the next word in the generated text
    :param probabilities: Probabilites of the next word
    :param int_to_vocab: Dictionary of word ids as the keys and words as the values
    :return: String of the predicted word
    """
    sampled_id=np.random.choice(len(probabilities), 1, p=probabilities)    
    return int_to_vocab[sampled_id[0]]

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_pick_word(pick_word)

Tests Passed


## Generate TV Script
This will generate the TV script for you.  Set `gen_length` to the length of TV script you want to generate.

In [45]:
gen_length = 200
# homer_simpson, moe_szyslak, or Barney_Gumble
prime_word = 'moe_szyslak'

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_dir + '.meta')
    loader.restore(sess, load_dir)

    # Get Tensors from loaded model
    input_text, initial_state, final_state, probs = get_tensors(loaded_graph)

    # Sentences generation setup
    gen_sentences = [prime_word + ':']
    prev_state = sess.run(initial_state, {input_text: np.array([[1]])})

    # Generate sentences
    for n in range(gen_length):
        # Dynamic Input
        dyn_input = [[vocab_to_int[word] for word in gen_sentences[-seq_length:]]]
        dyn_seq_length = len(dyn_input[0])

        # Get Prediction
        probabilities, prev_state = sess.run(
            [probs, final_state],
            {input_text: dyn_input, initial_state: prev_state})
        
        pred_word = pick_word(probabilities[dyn_seq_length-1], int_to_vocab)

        gen_sentences.append(pred_word)
    
    # Remove tokens
    tv_script = ' '.join(gen_sentences)
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        tv_script = tv_script.replace(' ' + token.lower(), key)
    tv_script = tv_script.replace('\n ', '\n')
    tv_script = tv_script.replace('( ', '(')
        
    print(tv_script)

moe_szyslak: an eighty-five yard field goal? you gotta be kidding me! refresh! refresh!
nfl_narrator:... no only never friend named...(pause) joey joe joe mural!
krusty_the_clown:(mad) mr. _zander: why i took away for a little dank.
duffman:(bitter) there you have worse. people drink an enemy. pick up!
book_club_member: anyone tries to stop me and wally to the bank. end up in a fevered pitch. a lot happier.
lenny:(carl voice) i'm just here, homer. you got change that drop?... one time at the treasure on edner, i... and" gay guide to springfield the springfield isotopes are time for hosting this." weekly reader star selection" an deal.
carl_carlson: well, we've all got that. i've been writin' creepy play.
man:(too innocent grampa) now what does your crew of the leave.
barney_gumble: be about pouring me...
moe_szyslak: who'da tell me how to


# The TV Script is Nonsensical
It's ok if the TV script doesn't make any sense.  We trained on less than a megabyte of text.  In order to get good results, you'll have to use a smaller vocabulary or get more data.  Luckily there's more data!  As we mentioned in the beggining of this project, this is a subset of [another dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data).  We didn't have you train on all the data, because that would take too long.  However, you are free to train your neural network on all the data.  After you complete the project, of course.
# Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_tv_script_generation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.